# Loop over simulation files + Analysis

In [1]:
import pyvista as pv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

## Read  Units (units.out)

In [2]:
units_data = pd.read_csv("./TURB_DRIVE_SUP/units.out")

rho_00   = np.array(units_data.loc[units_data["variable"] == "rho_0"]["normalisation"])
vel_00   = np.array(units_data.loc[units_data["variable"] == "v_0"]["normalisation"])
len_00   = np.array(units_data.loc[units_data["variable"] == "L_0"]["normalisation"])
c_iso    = np.array(units_data.loc[units_data["variable"] == "c_iso"]["normalisation"])

tim_00 = (len_00/vel_00)
bfi_00 = (np.sqrt(4*np.pi*rho_00*vel_00**2))

## Read  times (second column vtk.out)

In [6]:
time_data = pd.read_csv("./TURB_DRIVE_SUP/vtk.out", sep='\s+', header=None)

time_code = time_data.iloc[:,1]

time_cgs = time_code*tim_00

print(time_cgs)

0      0.000000e+00
1      5.614773e+10
2      1.123438e+11
3      1.828149e+11
4      2.443480e+11
           ...     
96     5.920976e+12
97     5.982550e+12
98     6.044594e+12
99     6.110076e+12
100    6.172000e+12
Name: 1, Length: 101, dtype: float64


In [11]:
print(np.max(time_cgs))

6172000000000.0


In [12]:
if os.path.isdir("./TURB_DRIVE_SUP/Figures1"):
    print("Directory already exists.")
else:
    print("Directory is being created.")
    os.mkdir("./TURB_DRIVE_SUP/Figures1")

Directory is being created.


In [14]:
### Create empty lists

t_list = []

av_list = []

std_list = []

rms_mach_list = []


for i in range(0,101):
    
    filename = "./TURB_DRIVE_SUP/data.0{:03d}.vtk".format(i) 
    
    #print(filename)
    
    # Opening the mesh
    mesh = pv.read(filename)
    
    # Getting the data arrays
    rho = pv.get_array(mesh, "rho", preference = 'cell') #density in code units
    vx1 = pv.get_array(mesh, "vx1", preference = 'cell') #vx in code units
    vx2 = pv.get_array(mesh, "vx2", preference = 'cell') #vy in code units    
    
    # Convert the arrays to 2D
    rho_2D = rho.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)
    vx1_2D = vx1.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)
    vx2_2D = vx2.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)
    
    # Convert arrays to CGS units
    rho_cgs2D = rho_2D*rho_00
    vx1_cgs2D = vx1_2D*vel_00
    vx2_cgs2D = vx2_2D*vel_00
    
    # Get i time
    time_i = time_cgs[i]
    
    
    # Diagnostics
    av_density  = np.mean(rho_cgs2D)
    std_density = np.std(rho_cgs2D)
    
    modulus_velocity = np.sqrt(vx1_cgs2D**2 + vx2_cgs2D**2)
    rms_mach = np.mean(modulus_velocity)/c_iso
    
    #print(av_density, std_density)
    
    # Fill up our empy lists
    
    t_list.append(time_i)
    av_list.append(av_density)
    std_list.append(std_density)
    rms_mach_list.append(rms_mach)
    
    # PLotting
    plt.figure(figsize = (16,10))
    
    plt.plot(t_list, std_list)
    
    plt.xlim(0, np.max(time_cgs))
    plt.ylim(1.e-26, 2.e-24)
    plt.savefig("./TURB_DRIVE_SUP/Figures1/std_dens{:03d}.png".format(i))
    plt.close()
    

    plt.figure(figsize = (16,10))
    
    plt.plot(t_list, rms_mach_list)
    
    plt.xlim(0, np.max(time_cgs))
    plt.ylim(0, 2)
    plt.savefig("./TURB_DRIVE_SUP/Figures1/rms_mach{:03d}.png".format(i))
    plt.close()
    
# Outside the loop, we create a data frame:

df = pd.DataFrame({'time': t_list, 'Av. Dens.': av_list, 'Std. Dens.': std_list})

df.to_csv("datos1.csv", sep=',', float_format='{:.2e}'.format)
print(df)

             time     Av. Dens.    Std. Dens.
0    0.000000e+00  1.660000e-24  3.673420e-40
1    5.614773e+10  1.660000e-24  4.354185e-27
2    1.123438e+11  1.660000e-24  1.526532e-26
3    1.828149e+11  1.660000e-24  3.950658e-26
4    2.443480e+11  1.660000e-24  7.787190e-26
..            ...           ...           ...
96   5.920976e+12  1.660000e-24  1.634459e-24
97   5.982550e+12  1.660000e-24  1.621632e-24
98   6.044594e+12  1.660000e-24  1.609173e-24
99   6.110076e+12  1.660000e-24  1.607816e-24
100  6.172000e+12  1.660000e-24  1.617887e-24

[101 rows x 3 columns]
